# 1 - Présentation
## 1.1 - Introduction
Les réseaux sociaux sont une source de données presques infinis. Avec les Tweets sur Twitter ou les murs sur Facebook, il y a une multitude d'informations en attente d'être étudiées.

L'objectif de ce jeu de données est de prédire si un tweet est positif ou négatif. L'exercice nécessite de réaliser une analyse de sentiment en réalisant une classification à 2 sorties.

## 1.2 - Source de Données
Les données sont représentés par 2 dimensions :
* un chiffre de 0 ou 1 qui représente si un tweet est positif(1) ou négatif(0) ;
* une chaine de caractères représentant le tweet.

# 2 - Préparation des données
## 2.1 - Charger les librairies
Ci-dessous la liste des librairies nécessaires durant l'analyse des données :

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

In [2]:
df = pd.read_csv('./data/training.csv', 
                 header=0,
                 sep=',',
                 names=['Sentimental','Id','Date','Query', 'Owner','Tweet'],
                 nrows = 1000,
                 encoding = 'utf8')

Maintenant nous allons afficher les 20 premières lignes de ce dataset pour mieux appréhender l'information :

In [3]:
df.head(20)

,Sentimental,Id,Date,Query,Owner,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1467812416,Mon Apr 06 22:20:16 PDT 2009,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


Il  y a 6 dimensions dans ce dataset :
* Le sentiment du tweet (0 = negatif, 2 = neutre et 4 = positif) ;
* L'id du tweet ;
* La date de publication ;
* La requête. (S'il n'y en a aucune alors NO_QUERY) ;
* L'utilisateur qui a tweeté ;
* Le texte du tweet.

Nous allons afficher les types des données chargées :

In [4]:
df.dtypes

Sentimental     int64
Id              int64
Date           object
Query          object
Owner          object
Tweet          object
dtype: object

Les types des données sont corrects pour l'analyse. Il n'est pas nécessaire de les casts.

## 2.3 - Nettoyer les données
Dans l'objectif d'analyser les données, il faut tout d'abord les nettoyer afin qu'elles soient plus facile à lire pour la machine.

Durant cette partie nous allons définir des règles pour le nettoyage de données non structurée au format texte.

In [18]:
def dataClean():
    signs = ['.', ',' ,'!', '?', ';', "'", '-', '_', '&', '(', ')', '*']
    
    df['Tweet'] = df['Tweet'].str.lower()
    df['Tweet'] = df['Tweet'].str.replace('@[\w]*', '')
    
    for sign in signs:
        df['Tweet'] = df['Tweet'].str.replace(sign, '')
    
    df['Tweet'] = df['Tweet'].str.replace('/^(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?$/', '')
    df['Tweet'] = df['Tweet'].str.strip()
    
dataClean()

Cette fonction va permettre de préparer les données pour la suite de l'analyse.

Les actions réalisées :
* Mettre le texte en minuscule afin d'éviter de compter certains mots en double comme par exemple : Hasard et hasard. C'est le même mot pour nous en le lisant mais pour l'algorithme cela est différent ;
* Supprimer les pseudos Twitter ;
* Supprimer les caractères comme le "." ou le "!" ;
* Supprimer les URL ;
* Supprimer les espaces avant et après le paragraphe.

## 2.4 - Preprocessing
Maintenant que nous avons nettoyer les données, il faut adapter l'information afin de la rendre exploitable par les algorithmes de Machine Learning.

In [ ]:
def preprocessing():
    df['TweetTK'] = df.apply(lambda row: nltk.word_tokenize(row['Tweet']), axis=1)
    df['TweetTK'] = df['TweetTK'].apply(lambda x: [item for item in x if item not in stopwords.words('english')])
    df['TweetTK'] = df['TweetTK'].apply(lambda x: [wordnet_lemmatizer.lemmatize(item) for item in x])

preprocessing()    

Cette fonction va permettre de **Tokenize** les tweets, faire disparaitre les **Stop Words** puis d'appliquer un algorithme **Lemmatisation**.

## 2.5 - Bag of Words

In [17]:
def getListofWords(tweets):
    allWords = []
    for words in tweets:
        allWords.extend(words)
    return allWords

df['TweetTK']
#test = getListofWords()
#test

KeyError: 'TweetTK'

## 2.6 - Conclusion
Les données sont maintenant exploitables pour commencer l'étape d'apprentissage en Machine Learning.

# 3. Apprentissage

In [ ]:
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

def find_features(document):
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features


tweets = []
for x, y in zip(df['TweetTK'], df['Sentimental']):
    tweets.append((x, y))


word_features = get_word_features(get_words_in_tweets(tweets))
word_features

In [ ]:
def extract_features(document):
    document_words = set(document)
    print(document)
    features = {}
    for word in tweet:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [ ]:
training_set = nltk.classify.apply_features(extract_features, tweets)

In [ ]:
print(training_set)

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
